Подготовка датасета

In [9]:
import os

import librosa
import pandas as pd

# Пути к файлам
dataset_path = "./data/normal_voices"
xlsx_path = "./data/Speeches.xlsx"

# Читаем данные из Excel
df = pd.read_excel(xlsx_path)

# Загружаем аудиофайлы и их MFCC
audio_data = {}
for _, row in df.iterrows():
    wav_file = row["file"]
    text = row["text"]
    audio_path = os.path.join(dataset_path, f"{wav_file}.wav")

    if os.path.exists(audio_path):
        y, sr = librosa.load(audio_path, sr=22050)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        audio_data[text] = (y, sr, mfcc)

/home/makarov/miniconda3/lib/python3.12/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(


Графовая модель на основе DTW

In [10]:

from fastdtw import fastdtw

# Функция поиска ближайшего совпадения
def find_closest_match(text_mfcc, audio_data):
    best_match = None
    best_dist = float("inf")

    for text, (y, sr, mfcc) in audio_data.items():
        dist, _ = fastdtw(text_mfcc.T, mfcc.T)
        if dist < best_dist:
            best_dist = dist
            best_match = y

    return best_match

# Генерация речи
def generate_speech(text, audio_data):
    # Если текст уже есть в аудиоданных, возвращаем его напрямую
    if text in audio_data:
        speech, sr, _ = audio_data[text]
    else:
        # Иначе ищем наиболее похожий фрагмент
        ref_text = list(audio_data.keys())[0]  # Выбираем любой реальный образец
        ref_audio, ref_sr, ref_mfcc = audio_data[ref_text]

        # Берем MFCC случайного реального аудиофайла (чтобы избежать случайного шума)
        text_mfcc = ref_mfcc
        speech = find_closest_match(text_mfcc, audio_data)

    # Сохраняем результат
    sf.write("output_dtw.wav", speech, 22050)

# Пример вызова
generate_speech("Привет", audio_data)


WaveNet с обучением с нуля

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import librosa
import os
import pandas as pd


# Читаем Excel-файл
df = pd.read_excel(xlsx_path)

# Модель WaveNet
class WaveNet(nn.Module):
    def __init__(self, n_input, n_output):
        super(WaveNet, self).__init__()
        # Первый слой: 128 каналов на входе, 32 канала на выходе
        self.conv1 = nn.Conv1d(n_input, 32, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        # Второй слой: 32 канала на входе, n_output на выходе
        self.conv2 = nn.Conv1d(32, n_output, kernel_size=3, padding=1)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.conv2(x)
        return x

# Генерация спектрограммы
def audio_to_mel(audio_path):
    y, sr = librosa.load(audio_path, sr=22050)
    mel = librosa.feature.melspectrogram(y=y, sr=sr)
    return mel

# Обучение модели
def train_wavenet(dataset_path, df):
    n_input = 128  # Размерность Mel-спектрограммы (каналы)
    n_output = 128  # Размерность выходных данных

    model = WaveNet(n_input=n_input, n_output=n_output)
    optimizer = optim.Adam(model.parameters())
    criterion = nn.MSELoss()

    for epoch in range(10):
        for _, row in df.iterrows():
            audio_path = os.path.join(dataset_path, f"{row['file']}.wav")

            if not os.path.exists(audio_path):
                continue  # Пропустить, если файл не найден

            mel = audio_to_mel(audio_path)
            mel = torch.tensor(mel).unsqueeze(0).float()  # Добавляем batch dimension: (1, freq, time)

            # Теперь меняем местами частоты и время, так как Conv1d ожидает (batch, channels, time)
            mel = mel.permute(0, 2, 1)  # (1, time, freq)

            output = model(mel)

            loss = criterion(output, mel)  # Выход должен совпадать с входом
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch}, Loss: {loss.item()}")

# Запуск обучения
train_wavenet(dataset_path, df)


RuntimeError: Given groups=1, weight of size [32, 128, 3], expected input[1, 58, 128] to have 128 channels, but got 58 channels instead

 LSTM + Spectrogram-based

In [ ]:
import torch
import torch.nn as nn
import librosa
import soundfile as sf

class TextToSpectrogramLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(TextToSpectrogramLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out

# Пример создания модели и тренировки
def train_lstm(dataset_path, df):
    model = TextToSpectrogramLSTM(input_size=256, hidden_size=512, output_size=128)
    optimizer = torch.optim.Adam(model.parameters())
    criterion = nn.MSELoss()

    # Тренировка
    for epoch in range(10):  # Количество эпох
        for _, row in df.iterrows():
            wav_file = row['file']
            text = row['text']

            # Преобразование аудио в спектрограмму
            mel = audio_to_mel(audio_path)
            mel = torch.tensor(mel).unsqueeze(0).float()

            # Прогон через модель
            output = model(mel)
            loss = criterion(output, mel)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch}, Loss: {loss.item()}")

train_lstm(dataset_path, df)
